# **대구 교통 사고 피해 예측 AI 경진대회 Baseline Code**

***

# Environment
- GPU 0  : NVIDIA GeForce RTX 3080 Ti Laptop GPU
- GPU 1  : AMD Radeon(TM) Graphics
- CPU : AMD Ryzen 9 6900HX with Radeon Graphics

In [1]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

fe = fm.FontEntry(fname = 'MaruBuri-Regular.otf', name = 'MaruBuri')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='MaruBuri')

In [2]:
import sys
import tqdm as tq
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np
print("-------------------------- Python & library version --------------------------")
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("tqdm version: {}".format(tq.__version__))
print("xgboost version: {}".format(xgb.__version__))
print("lightgbm version: {}".format(lgb.__version__))
print("catboost version: {}".format(cat.__version__))
print("seaborn version: {}".format(sns.__version__))
print("scikit-learn version: {}".format(skl.__version__))
print("------------------------------------------------------------------------------")

-------------------------- Python & library version --------------------------
Python version: 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
pandas version: 2.0.3
numpy version: 1.21.5
matplotlib version: 3.5.2
tqdm version: 4.64.1
xgboost version: 1.7.2
lightgbm version: 3.3.3
catboost version: 1.1.1
seaborn version: 0.11.2
scikit-learn version: 1.0.2
------------------------------------------------------------------------------


***

## **Fixed Random Seed**  

seed 값에 의해 동일한 코드를 사용해도 결과가 다를 수 있기에, 동일한 결과를 위해 seed 값을 고정시킵니다

In [ ]:
import os
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

***

## **데이터 불러오기 및 상위행 확인**  

train.csv, test.csv 파일을 로드하여 상위행을 출력해 봅시다

In [ ]:
import pandas as pd 

train_org = pd.read_csv('train.csv') 
test_org = pd.read_csv('test.csv')
countrywide_accident = pd.read_csv('./data/countrywide_accident.csv', encoding='utf-8')


In [ ]:
train_org = pd.concat([countrywide_accident,train_org],axis=0)
train_org = train_org.reset_index(drop=True)

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission.head()

***

## **데이터 전처리**  

현재 '사고일시', '시군구', '도로형태' 컬럼은 반복되는 패턴으로 여러 정보를 포함하고 있습니다
이런 반복되는 패턴을 일반화하면 pandas에서 제공하는 str.extract를 통해 한 번에 추출 가능합니다  

## **파생 변수 생성 1 : 날짜, 시간정보 생성**

'사고일시' 컬럼으로 부터 연도, 월, 일, 시간 정보 추출 및 변환 합니다

In [ ]:
train_org['사고일시'] = pd.to_datetime(train_org['사고일시'])
test_org['사고일시'] = pd.to_datetime(test_org['사고일시'])


In [ ]:
train_df = train_org.copy()
test_df = test_org.copy()

for df in [train_df, test_df]:
    df['연'] = df['사고일시'].dt.year
    df['월'] = df['사고일시'].dt.month
    df['일'] = df['사고일시'].dt.day
    df['monthday'] = df.apply(lambda row: str(row['월']) + '-' + str(row['일']), axis=1)
    df['시간'] = df['사고일시'].dt.hour
    df['weekday'] = df['사고일시'].dt.weekday
    df['weekofyear'] = (df['사고일시'].dt.isocalendar().week).astype(int)
    df['새벽'] = df['시간'].isin([0,1,2,3,4,5,6]).astype(int)
    df['밤'] = df['시간'].isin([21,22,23]).astype(int)
    df['주말'] = df['weekday'].isin([5,6]).astype(int)
    df['주중'] = df['weekday'].isin([0,1,2,3,4]).astype(int)
    df['국가공휴일'] = df['monthday'].isin(['1-1','3-1','5-5','6-6','8-15','10-3','10-9','12-25','12-31']).astype(int)
    df['covid-19'] = df['연'].apply(lambda x : 1 if x >= 2020
                                        else 0)

In [ ]:
#Time Cycling Transform 
##시간
train_df['sin_hour'] = np.sin(2 * np.pi * train_df['시간']/23.0)
train_df['cos_hour'] = np.cos(2 * np.pi * train_df['시간']/23.0)
test_df['sin_hour'] = np.sin(2 * np.pi * test_df['시간']/23.0)
test_df['cos_hour'] = np.cos(2 * np.pi * test_df['시간']/23.0)

##날짜
train_df['sin_date'] = -np.sin(2 * np.pi * (train_df['월']+train_df['일']/31)/12)
train_df['cos_date'] = -np.sin(2 * np.pi * (train_df['월']+train_df['일']/31)/12)
test_df['sin_date'] = -np.sin(2 * np.pi * (test_df['월']+test_df['일']/31)/12)
test_df['cos_date'] = -np.sin(2 * np.pi * (test_df['월']+test_df['일']/31)/12)

##월
train_df['sin_month'] = -np.sin(2 * np.pi * train_df['월']/12.0)
train_df['cos_month'] = -np.cos(2 * np.pi * train_df['월']/12.0)
test_df['sin_month'] = -np.sin(2 * np.pi * test_df['월']/12.0)
test_df['cos_month'] = -np.cos(2 * np.pi * test_df['월']/12.0)

In [ ]:
train_df = train_df.drop(columns=['사고일시','monthday']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다 
test_df = test_df.drop(columns=['사고일시','monthday'])


***

## **파생 변수 생성 2 : 공간(위치) 정보 생성** 

'시군구' 컬럼으로부터 의미 있는 공산 정보를 추출 및 변환 합니다

In [ ]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

In [ ]:
train_df.loc[(train_df['도시']=='울산광역시') & (train_df['구']=='중구'),'구']='울산중구'
train_df.loc[(train_df['도시']=='울산광역시') & (train_df['구']=='남구'),'구']='울산남구'
train_df.loc[(train_df['도시']=='울산광역시') & (train_df['구']=='동구'),'구']='울산동구'
train_df.loc[(train_df['도시']=='울산광역시') & (train_df['구']=='북구'),'구']='울산북구'

In [ ]:
train_df.loc[(train_df['도시']=='대전광역시') & (train_df['구']=='동구'),'구']='대전동구'
train_df.loc[(train_df['도시']=='대전광역시') & (train_df['구']=='중구'),'구']='대전중구'
train_df.loc[(train_df['도시']=='대전광역시') & (train_df['구']=='서구'),'구']='대전서구'

In [ ]:
train_df.loc[(train_df['도시']=='인천광역시') & (train_df['구']=='중구'),'구']='인천중구'
train_df.loc[(train_df['도시']=='인천광역시') & (train_df['구']=='동구'),'구']='인천동구'
train_df.loc[(train_df['도시']=='인천광역시') & (train_df['구']=='서구'),'구']='인천서구'


In [ ]:
train_df.loc[(train_df['도시']=='대구광역시') & (train_df['구']=='중구'),'구']='대구중구'
train_df.loc[(train_df['도시']=='대구광역시') & (train_df['구']=='동구'),'구']='대구동구'
train_df.loc[(train_df['도시']=='대구광역시') & (train_df['구']=='서구'),'구']='대구서구'
train_df.loc[(train_df['도시']=='대구광역시') & (train_df['구']=='남구'),'구']='대구남구'
train_df.loc[(train_df['도시']=='대구광역시') & (train_df['구']=='북구'),'구']='대구북구'

In [ ]:
train_df.loc[(train_df['도시']=='부산광역시') & (train_df['구']=='중구'),'구']='부산중구'
train_df.loc[(train_df['도시']=='부산광역시') & (train_df['구']=='동구'),'구']='부산동구'
train_df.loc[(train_df['도시']=='부산광역시') & (train_df['구']=='서구'),'구']='부산서구'
train_df.loc[(train_df['도시']=='부산광역시') & (train_df['구']=='남구'),'구']='부산남구'
train_df.loc[(train_df['도시']=='부산광역시') & (train_df['구']=='북구'),'구']='부산북구'

In [ ]:
train_df.loc[(train_df['도시']=='광주광역시') & (train_df['구']=='동구'),'구']='광주동구'
train_df.loc[(train_df['도시']=='광주광역시') & (train_df['구']=='서구'),'구']='광주서구'
train_df.loc[(train_df['도시']=='광주광역시') & (train_df['구']=='북구'),'구']='광주북구'
train_df.loc[(train_df['도시']=='광주광역시') & (train_df['구']=='남구'),'구']='광주남구'

In [ ]:
test_df.loc[(test_df['도시']=='대구광역시') & (test_df['구']=='중구'),'구']='대구중구'
test_df.loc[(test_df['도시']=='대구광역시') & (test_df['구']=='동구'),'구']='대구동구'
test_df.loc[(test_df['도시']=='대구광역시') & (test_df['구']=='서구'),'구']='대구서구'
test_df.loc[(test_df['도시']=='대구광역시') & (test_df['구']=='남구'),'구']='대구남구'
test_df.loc[(test_df['도시']=='대구광역시') & (test_df['구']=='북구'),'구']='대구북구'

In [ ]:
train_df.loc[(train_df['도시']=='서울특별시') & (train_df['구']=='강서구'),'구']='서울강서구'
train_df.loc[(train_df['도시']=='부산광역시') & (train_df['구']=='강서구'),'구']='부산강서구'

In [ ]:
train_df.loc[(train_df['도시']=='경상남도') & (train_df['구']=='고성군'),'구']='경상고성'
train_df.loc[(train_df['도시']=='강원도') & (train_df['구']=='고성군'),'구']='강원고성'

In [ ]:
train_df = train_df[~train_df['도시'].isna()].reset_index(drop=True)

***

# Feature Engineering

In [ ]:
k1 = train_df.groupby('구')['ECLO'].mean().reset_index()

In [ ]:
k2 = train_df.groupby('동')['ECLO'].mean().reset_index()

In [ ]:
고속도로1 = list(k1[k1['ECLO']>5]['구'])

In [ ]:
고속도로2 = list(k2[k2['ECLO']>5]['동'])

In [ ]:
train_df['고속도로여부1'] = train_df['구'].isin(고속도로1).astype(int)
test_df['고속도로여부1'] = test_df['구'].isin(고속도로1).astype(int)

train_df['고속도로여부2'] = train_df['동'].isin(고속도로2).astype(int)
test_df['고속도로여부2'] = test_df['동'].isin(고속도로2).astype(int)

In [ ]:
a1 = train_df.groupby('동')['사망자수'].sum().reset_index()
a2 = train_df.groupby('동')['중상자수'].sum().reset_index()
a3 = train_df.groupby('동')['경상자수'].sum().reset_index()
a4 = train_df.groupby('동')['부상자수'].sum().reset_index()

In [ ]:
a1.columns  = ['동','동사망자수']
a2.columns  = ['동','동중상자수']
a3.columns  = ['동','동경상자수']
a4.columns  = ['동','동부상자수']

In [ ]:
train_df = pd.merge(train_df, a1, how='left', on=['동'])
train_df = pd.merge(train_df, a2, how='left', on=['동'])
train_df = pd.merge(train_df, a3, how='left', on=['동'])
train_df = pd.merge(train_df, a4, how='left', on=['동'])

In [ ]:
test_df = pd.merge(test_df, a1, how='left', on=['동'])
test_df = pd.merge(test_df, a2, how='left', on=['동'])
test_df = pd.merge(test_df, a3, how='left', on=['동'])
test_df = pd.merge(test_df, a4, how='left', on=['동'])

In [ ]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_df['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_df['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

In [ ]:
train_df = train_df[train_df['도로형태2'] != '철길건널목'].reset_index(drop=True)

In [ ]:
train_df = train_df[train_df['기상상태'] != '안개'].reset_index(drop=True)

***

# MODELING

In [ ]:
train_df_1 = train_df.reset_index(drop=True)
test_df_1 = test_df

In [ ]:
test_x_1 = test_df_1.drop(columns=['ID']).copy()
train_x_1 = train_df_1[test_x_1.columns].copy()

train_y_1 = train_df_1['사망자수'].copy()
train_y_2 = train_df_1['중상자수'].copy()
train_y_3 = train_df_1['경상자수'].copy()
train_y_4 = train_df_1['부상자수'].copy()
train_y_5 = train_df_1['ECLO'].copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_features = list(train_x_1.dtypes[train_x_1.dtypes == "object"].index)
# 추출된 문자열 변수 확인

for i in categorical_features:
    le = LabelEncoder()
    le=le.fit(train_x_1[i]) 
    train_x_1[i]=le.transform(train_x_1[i])
    
    for case in np.unique(test_x_1[i]):
        if case not in le.classes_: 
            le.classes_ = np.append(le.classes_, case) 
    test_x_1[i]=le.transform(test_x_1[i])

from sklearn.preprocessing import LabelEncoder

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel


X = train_x_1
y = train_y_5

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,np.log1p(y), test_size=0.2, random_state=42)

# Create an XGBoost Regressor
model = XGBRegressor(
            max_depth=8,
            learning_rate=0.01,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=42,
            min_child_weight=50,
            objective='reg:squarederror',
            eval_metric='rmse')

model.fit(X_train, y_train)

# Display feature importances
feature_importances = model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
sel_features = feature_importance_df[feature_importance_df['Importance']>0]['Feature']

train_x_1 = train_x_1[sel_features]
test_x_1 = test_x_1[sel_features]

train_x_1

***

# XGBOOST

In [ ]:
iterations = 3000
patience = 100
is_holdout = False

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings(action='ignore')

# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')
models_1 = []
rmse_scores = []
n_split_list = [10,20]
for i in [0,11,25,1523,557,156,42,69,1125,5000]:
    for split in n_split_list:
        fold_idx = 1
        cv = StratifiedKFold(n_splits=split, shuffle=True, random_state=i)
        for train_index, valid_index in cv.split(train_x_1,train_y_5):
            X_train, X_valid = train_x_1.iloc[train_index], train_x_1.iloc[valid_index]
            Y_train, Y_valid = train_y_5[train_index], train_y_5[valid_index]
            log_Y_train, log_Y_valid = np.log1p(train_y_5[train_index]), np.log1p(train_y_5[valid_index])
            print("="*50)

            model = xgb.XGBRegressor(
                booster='gbtree',
                tree_method = 'gpu_hist',
                #device = 'gpu',
                n_estimators=iterations,
                max_depth=8,
                learning_rate=0.01,
                subsample=0.9,
                colsample_bytree=0.9,
                random_state=i,
                min_child_weight=50,
                objective='reg:squarederror',  # XGBoost에서 Tweedie 손실 함수
                eval_metric = 'rmse'
            )

            model.fit(
                X_train, log_Y_train,
                eval_set=[(X_valid, log_Y_valid)],
                early_stopping_rounds=patience,
                verbose=100
            )

            pred = model.predict(X_valid)

            models_1.append(model)
            fold_idx += 1
            if is_holdout:
                break



In [ ]:
preds_1  = []
for i in (range(300)):
    pred = models_1[i].predict(test_x_1)
    preds_1.append(np.expm1(pred))

preds_1 = np.mean(preds_1 , axis = 0)

***

# Make Submission

In [ ]:
baseline_submission = sample_submission.copy()
baseline_submission['ECLO'] = preds_1

In [ ]:
baseline_submission.to_csv('1209_ENS_SECOND.csv', index=False)

***

# ENSEMBLE WITH COLAB_FINAL

In [ ]:
import pandas as pd
answer1 = pd.read_csv('1209_ENS_SECOND.csv')
answer2 = pd.read_csv('COLAB_FINAL.csv')


In [ ]:
k1 = answer1['ECLO']* 0.2 + answer2['ECLO']*0.8 

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")

final_submission = sample_submission.copy()
final_submission['ECLO'] = k1

In [ ]:
final_submission.to_csv("private_검증용.csv", index=False)

***